In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
import joblib
import time
import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings('ignore')

# ML Libraries
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
import xgboost as xgb
from catboost import CatBoostClassifier

# Paths
BASE_DIR = Path(r'c:\Users\Acer\Desktop\Forex-Signal-App')
DATA_DIR = BASE_DIR / 'data'
MODEL_DIR = BASE_DIR / 'models' / 'signal_generator_v17'
MODEL_DIR.mkdir(parents=True, exist_ok=True)

# Constants
TIMEFRAMES = ['1min', '5min', '15min', '30min', '1h', '4h']
FORWARD_BARS = 15
TP_PIPS = 10
SL_PIPS = 10

print(f"🚀 FOREX SIGNAL V17: Hybrid Multi-Timeframe Ensemble")
print(f"✓ Model Directory: {MODEL_DIR}")

🚀 FOREX SIGNAL V17: Hybrid Multi-Timeframe Ensemble
✓ Model Directory: c:\Users\Acer\Desktop\Forex-Signal-App\models\signal_generator_v17


In [2]:
# 1. Load Data
def load_tf_data(timeframe, dataset_type='train'):
    filename = f'EURUSD_{timeframe}.csv'
    file_path = DATA_DIR / dataset_type / filename
        
    if not file_path.exists():
        print(f"⚠️ Warning: {file_path} not found.")
        return None
        
    df = pd.read_csv(file_path)
    df.columns = [c.lower() for c in df.columns]
    
    if 'time' in df.columns:
        df['time'] = pd.to_datetime(df['time'])
        df.set_index('time', inplace=True)
        df.sort_index(inplace=True)
    
    return df

timeframes_map = {
    '1min': 'm1',
    '5min': 'm5',
    '15min': 'm15',
    '30min': 'm30',
    '1h': 'h1',
    '4h': 'h4'
}

dfs_train = {}
dfs_test = {}

print("Loading TRAIN data...")
for tf_name, tf_code in timeframes_map.items():
    dfs_train[tf_name] = load_tf_data(tf_code, 'train')
    if dfs_train[tf_name] is not None:
        print(f"  {tf_name}: {len(dfs_train[tf_name]):,} rows")

print("\nLoading TEST data...")
for tf_name, tf_code in timeframes_map.items():
    dfs_test[tf_name] = load_tf_data(tf_code, 'test')
    if dfs_test[tf_name] is not None:
        print(f"  {tf_name}: {len(dfs_test[tf_name]):,} rows")

Loading TRAIN data...
  1min: 3,354,904 rows
  5min: 671,581 rows
  15min: 224,382 rows
  30min: 112,194 rows
  1h: 56,098 rows
  4h: 14,498 rows

Loading TEST data...
  1min: 743,476 rows
  5min: 148,502 rows
  15min: 49,807 rows
  30min: 24,907 rows
  1h: 12,454 rows
  4h: 3,220 rows


In [3]:
# 2. Enhanced Feature Engineering (V10 Style)
def add_features(df):
    if df is None or len(df) == 0:
        return None
    df = df.copy()
    
    # --- Time Features ---
    df['hour'] = df.index.hour
    df['day_of_week'] = df.index.dayofweek
    df['is_london'] = ((df['hour'] >= 8) & (df['hour'] < 16)).astype(int)
    df['is_ny'] = ((df['hour'] >= 13) & (df['hour'] < 21)).astype(int)
    
    # --- Moving Averages ---
    for p in [9, 21, 50, 200]:
        df[f'ema_{p}'] = df['close'].ewm(span=p, adjust=False).mean()
        # Distance from EMA
        df[f'dist_ema_{p}'] = (df['close'] - df[f'ema_{p}']) / df[f'ema_{p}']
        
    # --- RSI ---
    delta = df['close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / (loss + 1e-10)
    df['rsi'] = 100 - (100 / (1 + rs))
    
    # --- MACD ---
    ema12 = df['close'].ewm(span=12, adjust=False).mean()
    ema26 = df['close'].ewm(span=26, adjust=False).mean()
    df['macd'] = ema12 - ema26
    df['macd_signal'] = df['macd'].ewm(span=9, adjust=False).mean()
    df['macd_hist'] = df['macd'] - df['macd_signal']
    
    # --- Bollinger Bands ---
    df['bb_mid'] = df['close'].rolling(window=20).mean()
    df['bb_std'] = df['close'].rolling(window=20).std()
    df['bb_upper'] = df['bb_mid'] + (df['bb_std'] * 2)
    df['bb_lower'] = df['bb_mid'] - (df['bb_std'] * 2)
    df['bb_width'] = (df['bb_upper'] - df['bb_lower']) / df['bb_mid']
    df['bb_pos'] = (df['close'] - df['bb_lower']) / (df['bb_upper'] - df['bb_lower'] + 1e-10)
    
    # --- ATR & Volatility ---
    high_low = df['high'] - df['low']
    high_close = (df['high'] - df['close'].shift()).abs()
    low_close = (df['low'] - df['close'].shift()).abs()
    ranges = pd.concat([high_low, high_close, low_close], axis=1)
    true_range = ranges.max(axis=1)
    df['atr'] = true_range.rolling(14).mean()
    df['volatility'] = df['close'].rolling(20).std()
    
    # --- Price Action ---
    df['body_size'] = (df['close'] - df['open']).abs()
    df['upper_shadow'] = df['high'] - df[['close', 'open']].max(axis=1)
    df['lower_shadow'] = df[['close', 'open']].min(axis=1) - df['low']
    
    return df.dropna()

print("Processing TRAIN features...")
processed_train = {k: add_features(v) for k, v in dfs_train.items()}

print("Processing TEST features...")
processed_test = {k: add_features(v) for k, v in dfs_test.items()}

print("Feature engineering complete.")

Processing TRAIN features...
Processing TEST features...
Feature engineering complete.


In [4]:
# 3. Target Creation
def create_target(df, tp_pips=10, sl_pips=10, forward_bars=15):
    targets = []
    pip_size = 0.0001
    tp = tp_pips * pip_size
    sl = sl_pips * pip_size
    
    closes = df['close'].values
    highs = df['high'].values
    lows = df['low'].values
    
    for i in range(len(df) - forward_bars):
        current_close = closes[i]
        future_highs = highs[i+1 : i+1+forward_bars]
        future_lows = lows[i+1 : i+1+forward_bars]
        
        max_high = np.max(future_highs)
        min_low = np.min(future_lows)
        
        if (max_high - current_close) >= tp and (current_close - min_low) < sl:
            targets.append(1) # BUY
        elif (current_close - min_low) >= tp and (max_high - current_close) < sl:
            targets.append(2) # SELL
        else:
            targets.append(0) # HOLD
            
    targets.extend([0] * forward_bars)
    return np.array(targets)

print("Creating targets...")
train_datasets = {}
test_datasets = {}

for tf, df in processed_train.items():
    if df is not None and len(df) > FORWARD_BARS:
        df = df.copy()
        df['target'] = create_target(df, TP_PIPS, SL_PIPS, FORWARD_BARS)
        train_datasets[tf] = df.iloc[:-FORWARD_BARS]

for tf, df in processed_test.items():
    if df is not None and len(df) > FORWARD_BARS:
        df = df.copy()
        df['target'] = create_target(df, TP_PIPS, SL_PIPS, FORWARD_BARS)
        test_datasets[tf] = df.iloc[:-FORWARD_BARS]

print("Target creation complete.")

Creating targets...
Target creation complete.


In [5]:
# 4. Training with Class Balancing
models = {}
scalers = {}
feature_cols_dict = {}

for tf, df_train in train_datasets.items():
    print(f"\nTraining Model for {tf}...")
    
    # Features
    cols = [c for c in df_train.columns if c not in ['target', 'open', 'high', 'low', 'close', 'tick_volume', 'spread', 'volume']]
    
    # --- Undersampling Majority Class (HOLD) ---
    # Separate classes
    df_hold = df_train[df_train['target'] == 0]
    df_buy = df_train[df_train['target'] == 1]
    df_sell = df_train[df_train['target'] == 2]
    
    # Undersample HOLD to match 2x the size of minority classes (to keep some context but reduce dominance)
    n_minority = max(len(df_buy), len(df_sell))
    if len(df_hold) > n_minority * 3:
        df_hold_downsampled = resample(df_hold, 
                                       replace=False,    # sample without replacement
                                       n_samples=n_minority * 3, # match minority size * 3
                                       random_state=42)
    else:
        df_hold_downsampled = df_hold
        
    # Combine back
    df_balanced = pd.concat([df_hold_downsampled, df_buy, df_sell])
    
    X_train = df_balanced[cols]
    y_train = df_balanced['target']
    
    print(f"  Original Size: {len(df_train)}, Balanced Size: {len(df_balanced)}")
    print(f"  Class Dist: {y_train.value_counts().to_dict()}")
    
    # Scale
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    
    # Train XGBoost with scale_pos_weight equivalent logic
    # Since we undersampled, we might not need extreme weights, but let's use a balanced approach
    model = xgb.XGBClassifier(
        n_estimators=800,
        learning_rate=0.03,
        max_depth=7,
        subsample=0.8,
        colsample_bytree=0.8,
        objective='multi:softprob',
        num_class=3,
        random_state=42,
        n_jobs=-1,
        verbosity=0
    )
    model.fit(X_train_scaled, y_train)
    
    models[tf] = model
    scalers[tf] = scaler
    feature_cols_dict[tf] = cols
    
    print(f"  ✓ {tf} Model Trained.")

print("\nAll models trained successfully.")


Training Model for 1min...
  Original Size: 3354870, Balanced Size: 1136567
  Class Dist: {0: 684057, 2: 228019, 1: 224491}
  ✓ 1min Model Trained.

Training Model for 5min...
  Original Size: 671547, Balanced Size: 671547
  Class Dist: {0: 359207, 2: 157239, 1: 155101}
  ✓ 5min Model Trained.

Training Model for 15min...
  Original Size: 224348, Balanced Size: 224348
  Class Dist: {0: 85696, 2: 69358, 1: 69294}
  ✓ 15min Model Trained.

Training Model for 30min...
  Original Size: 112160, Balanced Size: 112160
  Class Dist: {0: 45951, 1: 33386, 2: 32823}
  ✓ 30min Model Trained.

Training Model for 1h...
  Original Size: 56064, Balanced Size: 56064
  Class Dist: {0: 30945, 1: 12663, 2: 12456}
  ✓ 1h Model Trained.

Training Model for 4h...
  Original Size: 14464, Balanced Size: 7879
  Class Dist: {0: 4752, 2: 1584, 1: 1543}
  ✓ 4h Model Trained.

All models trained successfully.


In [6]:
# 5. Evaluation
def get_predictions(models, scalers, datasets):
    preds_dict = {}
    for tf, df in datasets.items():
        if tf in models:
            model = models[tf]
            scaler = scalers[tf]
            cols = feature_cols_dict[tf]
            
            X = df[cols]
            X_scaled = scaler.transform(X)
            
            probs = model.predict_proba(X_scaled)
            preds_df = pd.DataFrame(probs, columns=['HOLD', 'BUY', 'SELL'], index=df.index)
            preds_dict[tf] = preds_df
    return preds_dict

def evaluate_ensemble(preds_dict, datasets, name="Set"):
    base_tf = '1min'
    if base_tf not in datasets:
        print(f"Error: 1min data needed for {name} evaluation.")
        return

    base_index = datasets[base_tf].index
    agg_probs = pd.DataFrame(0.0, index=base_index, columns=['HOLD', 'BUY', 'SELL'])
    
    count = 0
    for tf, preds_df in preds_dict.items():
        aligned_preds = preds_df.reindex(base_index, method='ffill')
        agg_probs = agg_probs.add(aligned_preds, fill_value=0)
        count += 1
        
    final_probs = agg_probs / count
    final_classes = final_probs.idxmax(axis=1)
    
    class_map = {'HOLD': 0, 'BUY': 1, 'SELL': 2}
    y_pred = final_classes.map(class_map).fillna(0).astype(int)
    y_true = datasets[base_tf]['target'].reindex(base_index).fillna(0).astype(int)
    
    print(f"\n=== {name} Ensemble Performance ===")
    print(f"Overall Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(classification_report(y_true, y_pred))
    
    # --- Trade Only Evaluation ---
    trade_mask = y_pred != 0
    if trade_mask.sum() > 0:
        trade_preds = y_pred[trade_mask]
        trade_actuals = y_true[trade_mask]
        
        print(f"\n=== {name} Trade Only Performance (Excluding HOLD) ===")
        print(f"Total Trades: {len(trade_preds)}")
        print(f"Trade Accuracy: {accuracy_score(trade_actuals, trade_preds):.4f}")
        print(classification_report(trade_actuals, trade_preds, zero_division=0))
    else:
        print(f"\n=== {name} Trade Only Performance ===")
        print("No trades predicted.")

print("Generating predictions for TEST set...")
test_preds = get_predictions(models, scalers, test_datasets)
evaluate_ensemble(test_preds, test_datasets, "TEST")

Generating predictions for TEST set...

=== TEST Ensemble Performance ===
Overall Accuracy: 0.8886
              precision    recall  f1-score   support

           0       0.91      0.98      0.94    674901
           1       0.10      0.05      0.07     33974
           2       0.15      0.02      0.03     34567

    accuracy                           0.89    743442
   macro avg       0.39      0.35      0.35    743442
weighted avg       0.84      0.89      0.86    743442


=== TEST Trade Only Performance (Excluding HOLD) ===
Total Trades: 21892
Trade Accuracy: 0.1126
              precision    recall  f1-score   support

           0       0.00      0.00      0.00     16771
           1       0.10      0.78      0.18      2392
           2       0.15      0.22      0.18      2729

    accuracy                           0.11     21892
   macro avg       0.08      0.33      0.12     21892
weighted avg       0.03      0.11      0.04     21892



In [7]:
# 6. Save Models
print("Saving models...")
joblib.dump(models, MODEL_DIR / 'models_dict_v17.joblib')
joblib.dump(scalers, MODEL_DIR / 'scalers_dict_v17.joblib')
joblib.dump(feature_cols_dict, MODEL_DIR / 'feature_cols_dict_v17.joblib')

config = {
    'timeframes': TIMEFRAMES,
    'tp_pips': TP_PIPS,
    'sl_pips': SL_PIPS,
    'forward_bars': FORWARD_BARS,
    'strategy': 'hybrid_ensemble_undersampling'
}
joblib.dump(config, MODEL_DIR / 'config_v17.joblib')

print(f"Models saved to {MODEL_DIR}")

Saving models...
Models saved to c:\Users\Acer\Desktop\Forex-Signal-App\models\signal_generator_v17
